In [35]:
import requests
import json
import pandas as pd

country_code = "MN"  # ISO 3166-1 alpha-2 country code for Montenegro
parameter = "pm25"  # can also use the parameter id, 2

url = f"https://api.openaq.org/v2/locations?country={country_code}&parameter={parameter}"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()  # Parse the JSON response
    locations = data['results']  # Get the list of locations

    # Create an empty DataFrame
    df = pd.DataFrame(columns=["Location ID", "City", "Name", "Coordinates"])

    # Extract and store the data in the DataFrame
    for location in locations:
        location_id = location['id']
        city = location['city']
        name = location['name']
        coordinates = location['coordinates']
        first_time_sensor_reading = location["parameters"][0]["firstUpdated"]
        df = df.append({"Location ID": location_id, "City": city, "Name": name, "Coordinates": coordinates, "Start Date": first_time_sensor_reading}, ignore_index=True)

    # Print the DataFrame or save it to a file
    df.to_csv('Data/OpenAQI_Mongolia_Sensors.csv', index=False)

    # If you want to save the DataFrame to a CSV file:
    # df.to_csv('location_data.csv', index=False)
else:
    print(f"Error: {response.status_code} - Unable to fetch data from the API")

/var/folders/5b/1b64wf4d3lsfd20lg09_h0xh0000gp/T/ipykernel_29248/3653149612.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Location ID": location_id, "City": city, "Name": name, "Coordinates": coordinates, "Start Date": first_time_sensor_reading}, ignore_index=True)
/var/folders/5b/1b64wf4d3lsfd20lg09_h0xh0000gp/T/ipykernel_29248/3653149612.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Location ID": location_id, "City": city, "Name": name, "Coordinates": coordinates, "Start Date": first_time_sensor_reading}, ignore_index=True)
/var/folders/5b/1b64wf4d3lsfd20lg09_h0xh0000gp/T/ipykernel_29248/3653149612.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Locatio

In [38]:
import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import Point
import ast

# Create GeoPandas GeoDataFrame
geometry = [Point(coords['longitude'], coords['latitude']) for coords in df['Coordinates']]
OpenAQI_MN = gpd.GeoDataFrame(df, geometry=geometry)

# Drop a column by name
OpenAQI_MN = OpenAQI_MN.drop("Coordinates", axis=1)
OpenAQI_MN = OpenAQI_MN.drop_duplicates()

print(len(OpenAQI_MN["Location ID"].unique()))

OpenAQI_MN

91


,Location ID,City,Name,Start Date,geometry
0,525529,None,Zavkhan_01,2023-01-26T08:51:54+00:00,POINT (96.84302 47.74261)
1,230487,None,Dornod_01(Цаг уур),2023-06-07T15:34:12+00:00,POINT (114.53722 48.08028)
2,227901,None,NAMEM Site near KG200,2023-05-23T17:57:18+00:00,POINT (107.11800 47.86650)
3,525536,None,Arkhangai,2023-01-26T08:49:40+00:00,POINT (101.46247 47.47228)
4,230697,None,Uburkhangai_01( Хас банк),2023-05-23T17:58:14+00:00,POINT (102.77983 46.26303)
...,...,...,...,...,...
86,48,None,Baruun 4 zam,2016-01-30T00:15:00+00:00,POINT (106.89419 47.91538)
87,6523,None,1-r khoroolol,2019-03-01T07:45:00+00:00,POINT (106.84806 47.91798)
88,2275,None,Bayankhoshuu,2016-05-25T09:00:00+00:00,POINT (106.82275 47.95756)
89,6489,None,1-r khoroolol,2019-03-01T06:45:00+00:00,POINT (106.50530 47.55047)


In [47]:
mng_bd = gpd.read_file("/Users/turbold/Documents/Projects/Green Summer DS/mng-administrative-divisions-shapefiles/mng_admbnda_adm1_nso_20201019.shp")

mng_bd

import geopandas as gpd
from shapely.geometry import Point
from geopy.geocoders import Nominatim

def get_nearest_city_name(point):
    geolocator = Nominatim(user_agent="nearest_city_locator")
    location = geolocator.reverse((point.y, point.x), exactly_one=True)
    return location.raw.get('address', {}).get('city', None)

def get_full_address(point):
    geolocator = Nominatim(user_agent="full_address_locator")
    location = geolocator.reverse((point.y, point.x), language="en", addressdetails=True)
    return location.raw.get('display_name', None)

# Assuming your GeoDataFrame is named 'gdf'
OpenAQI_MN['address'] = OpenAQI_MN['geometry'].apply(get_full_address)


In [48]:
OpenAQI_MN

,Location ID,City,Name,Start Date,geometry,nearest_city,address
0,525529,None,Zavkhan_01,2023-01-26T08:51:54+00:00,POINT (96.84302 47.74261),Улиастай,"Uliastai, Zavkhan, Mongolia"
1,230487,None,Dornod_01(Цаг уур),2023-06-07T15:34:12+00:00,POINT (114.53722 48.08028),Чойбалсан,"10 -Р БАГ, Цаг уур, Choibalsan, Kherlen, Dorno..."
2,227901,None,NAMEM Site near KG200,2023-05-23T17:57:18+00:00,POINT (107.11800 47.86650),Улаанбаатар ᠤᠯᠠᠭᠠᠨᠪᠠᠭᠠᠲᠤᠷ,"Bayanzürkh, Ulaanbaatar, Mongolia"
3,525536,None,Arkhangai,2023-01-26T08:49:40+00:00,POINT (101.46247 47.47228),Эрдэнэбулган ᠡᠷᠳᠡᠨᠢᠪᠤᠯᠠᠭᠠᠨ,"Erdenebulgan, Arkhangai, 65000, Mongolia"
4,230697,None,Uburkhangai_01( Хас банк),2023-05-23T17:58:14+00:00,POINT (102.77983 46.26303),Арвайхээр ᠠᠷᠪᠠᠶᠢᠬᠡᠭᠡᠷ,"Renchinpelje Street, Arvaikheer, Övörkhangai, ..."
...,...,...,...,...,...,...,...
86,48,None,Baruun 4 zam,2016-01-30T00:15:00+00:00,POINT (106.89419 47.91538),Улаанбаатар ᠤᠯᠠᠭᠠᠨᠪᠠᠭᠠᠲᠤᠷ,"Барилгын коллеж, Peace Avenue, Gandan, Bayango..."
87,6523,None,1-r khoroolol,2019-03-01T07:45:00+00:00,POINT (106.84806 47.91798),Улан-Батор,"72а, Хайрхан-3, 1-р хороолол, Songino Khairkha..."
88,2275,None,Bayankhoshuu,2016-05-25T09:00:00+00:00,POINT (106.82275 47.95756),Улаанбаатар ᠤᠯᠠᠭᠠᠨᠪᠠᠭᠠᠲᠤᠷ,"Bayankhoshuu, Songino Khairkhan, Ulaanbaatar, ..."
89,6489,None,1-r khoroolol,2019-03-01T06:45:00+00:00,POINT (106.50530 47.55047),None,"Altanbulag, Töv, Mongolia"


In [6]:
##### Creates Interactive HTML Map to show locations of Sensors #######

# Create an interactive map using Folium
m = folium.Map(location=[47.5, 102.5], zoom_start=5)

# Plot the GeoPandas DataFrame on the map
for idx, row in gdf.iterrows():
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=row["City"],
    ).add_to(m)

# Display the interactive map
m.save("locations_map.html")  # Save the map to an HTML file